Kien Truong
<br>
CPSC 483-02
<br>
Professor Avery
<br>
Due date: 5/7/2021
<br>
$$ Project #6 description $$
<br>
This project works with a "real-world" dataset using UC Irvine repository and examines how an imbalanced dataset can influence classifier performance.

In [1]:
import pandas as pd
import numpy as np
'''
Question 2 - Load data
'''
training = pd.read_csv("bank-additional.csv", sep=";")
test = pd.read_csv("bank-additional-full.csv", sep=";")
training

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4114,30,admin.,married,basic.6y,no,yes,yes,cellular,jul,thu,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.958,5228.1,no
4115,39,admin.,married,high.school,no,yes,no,telephone,jul,fri,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.959,5228.1,no
4116,27,student,single,high.school,no,no,no,cellular,may,mon,...,2,999,1,failure,-1.8,92.893,-46.2,1.354,5099.1,no
4117,58,admin.,married,high.school,no,no,no,cellular,aug,fri,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.966,5228.1,no


In [2]:
'''
Question 3 - Drop "duration" column, separate features and targets, get dummy coding, and standardize features
'''
from sklearn.preprocessing import StandardScaler

#Drop 'duration' attribute from the training a test sets
training = training.drop(columns="duration")
test = test.drop(columns="duration")

#Separate the features from the target 'y'-the last column
X_training = training.iloc[:, 0:19:1]
t_training = training.iloc[:, -1]
X_test = test.iloc[:, 0:19:1]
t_test = test.iloc[:, -1]

#Get dummy coding
X_training = pd.get_dummies(X_training, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)
t_training = np.array(pd.get_dummies(t_training, drop_first=True))
t_test = np.array(pd.get_dummies(t_test, drop_first=True))
# print(t_test)

#Standardize features
scaler = StandardScaler()
fit_training = scaler.fit(X_training)
standardized_X_training = scaler.transform(X_training)
# X_training = scaler.transform(X_training)

fit_test = scaler.fit(X_test)
standardized_X_test = scaler.transform(X_test)
# X_test = scaler.transform(X_test)

In [3]:
'''
Question 4 - Fit Naive Bayes, KNN, and SVM classifiers to the training set, then score each classifier on the tet set.
'''
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

fit_NB = GaussianNB().fit(X_training, t_training.flatten())
score_NB = fit_NB.score(X_test, t_test)
print(f"The score value using Naive Bayes classifier is: {score_NB}")

fit_KNN = KNeighborsClassifier().fit(standardized_X_training, t_training.flatten())
score_KNN = fit_KNN.score(standardized_X_test, t_test)
print(f"The score value using KNN classifier is: {score_KNN}")

fit_SVM = SVC().fit(standardized_X_training, t_training.flatten())
score_SVM = fit_SVM.score(standardized_X_test, t_test)
print(f"The score value using SVM classifier is: {score_SVM}")

The score value using Naive Bayes classifier is: 0.8545450131106147
The score value using KNN classifier is: 0.8938040205885209
The score value using SVM classifier is: 0.8981985044187627


The SVM classifier has the highest accuracy among the three classifiers.

In [4]:
'''
Question 5 - how many y_yes = 0 and y_yes = 1?
'''
yes_0 = len(np.where(t_training == 0)[0])
yes_1 = len(np.where(t_training == 1)[0])
print(f"There are {yes_0} values in the training set have y_yes = 0 and {yes_1} values have y_yes = 1")

There are 3668 values in the training set have y_yes = 0 and 451 values have y_yes = 1


If we assumed that no customer ever subscribed to the product, then the accuracy would be close to 99% because it always predicts class y_yes = 0 without looking at all the feature vectors.

In [5]:
'''
Question 6 - Create a target vector representing the dumn classifier, then create a confusion matrix and find its AUC
'''
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

t_pred = np.zeros_like(t_training)
conf_matrix = confusion_matrix(t_training, t_pred)
print(f"The confusion matrix is\n {conf_matrix}")
auc_score = roc_auc_score(t_training, t_pred)
print(f"The AUC score for the dumb classifier is {auc_score}")

The confusion matrix is
 [[3668    0]
 [ 451    0]]
The AUC score for the dumb classifier is 0.5


In [6]:
'''
Question 7 - Create a confusion matrix and find the AUC for each classifiers.
'''
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

'''
----- Naive Bayes -----
'''
t_naive_pred = fit_NB.predict(X_test)
naive_conf_matrix = confusion_matrix(t_test, t_naive_pred)
print(f"The confusion matrix for Naive Bayes is\n {naive_conf_matrix}")
naive_auc_score = roc_auc_score(t_test, t_naive_pred)
print(f"The AUC score for Naive Bayes classifier is {naive_auc_score}")
'''
----- KNN -----
'''
t_knn_pred = fit_KNN.predict(standardized_X_test)
knn_conf_matrix = confusion_matrix(t_test, t_knn_pred)
print(f"The confusion matrix for KNN is\n {knn_conf_matrix}")
knn_auc_score = roc_auc_score(t_test, t_knn_pred)
print(f"The AUC score for KNN classifier is {knn_auc_score}")
'''
----- SVM -----
'''
t_svm_pred = fit_SVM.predict(standardized_X_test)
svm_conf_matrix = confusion_matrix(t_test, t_svm_pred)
print(f"The confusion matrix for SVM is\n {svm_conf_matrix}")
svm_auc_score = roc_auc_score(t_test, t_svm_pred)
print(f"The AUC score for SVM classifier is {svm_auc_score}")

The confusion matrix for Naive Bayes is
 [[33020  3528]
 [ 2463  2177]]
The AUC score for Naive Bayes classifier is 0.6863252222867992
The confusion matrix for KNN is
 [[35674   874]
 [ 3500  1140]]
The AUC score for KNN classifier is 0.6108879489608374
The confusion matrix for SVM is
 [[36193   355]
 [ 3838   802]]
The AUC score for SVM classifier is 0.5815657868914946


The Naive Bayes classifier has the highest accuracy rate; therefore, it is the best classifier.

In [11]:
'''
Question 8 - Use fit_resample() to generate a balanced training set.
'''
from imblearn.over_sampling import RandomOverSampler

#Separate the features from the target 'y'-the last column
X_training = training.iloc[:, 0:19:1]
t_training = training.iloc[:, -1]
X_test = test.iloc[:, 0:19:1]
t_test = test.iloc[:, -1]
# print(X_training.shape)

#Generate balanced training set
ros = RandomOverSampler(random_state=2021-4-22)
bal_X_training, bal_t_training = ros.fit_resample(X_training, t_training)
# bal_X_test, bal_t_test = ros.fit_resample(X_test, t_test)
# print(bal_X_training)

#Get dummy coding
bal_X_training = pd.get_dummies(bal_X_training, drop_first=True)
bal_t_training = np.array(pd.get_dummies(bal_t_training, drop_first=True))
X_test = pd.get_dummies(X_test, drop_first=True)
t_test = np.array(pd.get_dummies(t_test, drop_first=True))
# X_test = pd.get_dummies(bal_X_test, drop_first=True)

In [12]:
'''
Question 9 - Repeat experiments 4 and 7 on the balanced training set of experiment 8. Which classifier 
             performs the best?
'''
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

'''
----- Fit the training and score each classifier on the test set -----
'''
fit_NB = GaussianNB().fit(bal_X_training, bal_t_training.flatten())
score_NB = fit_NB.score(X_test, t_test)
print(f"The score value using Naive Bayes classifier is: {score_NB}")

fit_KNN = KNeighborsClassifier().fit(bal_X_training, bal_t_training.flatten())
score_KNN = fit_KNN.score(X_test, t_test)
print(f"The score value using KNN classifier is: {score_KNN}")

fit_SVM = SVC().fit(bal_X_training, bal_t_training.flatten())
score_SVM = fit_SVM.score(X_test, t_test)
print(f"The score value using SVM classifier is: {score_SVM}\n")

'''
----- Create a confusion matrix and find the AUC for each classifier -----
'''
t_naive_pred = fit_NB.predict(X_test)
naive_conf_matrix = confusion_matrix(t_test, t_naive_pred)
naive_auc_score = roc_auc_score(t_test, t_naive_pred)
print(f"The confusion matrix for Naive Bayes is\n {naive_conf_matrix}")
print(f"The AUC score for Naive Bayes classifier is {naive_auc_score}")

t_knn_pred = fit_KNN.predict(X_test)
knn_conf_matrix = confusion_matrix(t_test, t_knn_pred)
knn_auc_score = roc_auc_score(t_test, t_knn_pred)
print(f"The confusion matrix for KNN is\n {knn_conf_matrix}")
print(f"The AUC score for KNN classifier is {knn_auc_score}")

t_svm_pred = fit_SVM.predict(X_test)
svm_conf_matrix = confusion_matrix(t_test, t_svm_pred)
svm_auc_score = roc_auc_score(t_test, t_svm_pred)
print(f"The confusion matrix for SVM is\n {svm_conf_matrix}")
print(f"The AUC score for SVM classifier is {svm_auc_score}")

The score value using Naive Bayes classifier is: 0.8446392153054287
The score value using KNN classifier is: 0.772093813732155
The score value using SVM classifier is: 0.8720986695153928

The confusion matrix for Naive Bayes is
 [[32481  4067]
 [ 2332  2308]]
The AUC score for Naive Bayes classifier is 0.6930677370901941
The confusion matrix for KNN is
 [[29073  7475]
 [ 1912  2728]]
The AUC score for KNN classifier is 0.6917027395244044
The confusion matrix for SVM is
 [[33617  2931]
 [ 2337  2303]]
The AUC score for SVM classifier is 0.7080701500718942


The SVM classifier performs the best because it has the highest AUC score of 0.708 and the highest score value of 0.872 compared to the remaining classifiers. 